In [1]:
import pandas as pd
from datetime import datetime

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

## Carga de los dataset necesarios

In [2]:
df_reviews = pd.read_csv('data/df_reviews_unido.csv')

In [3]:
df_reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,items_count,price,genres,release_anio,developer,playtime_horas
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,277.0,19.99,Action,2009,Tripwire Interactive,5060.666667
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,1,888.0,NaN,Sin dato disponible,Sin dato disponible,Sin dato disponible,5539.916667
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Action,2013,"Hopoo Games, LLC",6180.266667
3,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Indie,2013,"Hopoo Games, LLC",6180.266667
4,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,RPG,2013,"Hopoo Games, LLC",6180.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
130628,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Casual,2015,Trion Worlds,1862.433333
130629,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Free to Play,2015,Trion Worlds,1862.433333
130630,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,Massively Multiplayer,2015,Trion Worlds,1862.433333
130631,lifeonhigh,http://steamcommunity.com/id/lifeonhigh,304050,No ratings yet,True,2015-12-20,1,45.0,0.00,RPG,2015,Trion Worlds,1862.433333


In [7]:
genre_ranking = pd.read_csv('data/df_genre_ranking_unido.csv')

In [40]:
df_items_developer = pd.read_csv('data/df_items_developer_unido.csv')

In [41]:
df_items_developer

,item_id,price,developer,release_anio
0,10,9.99,Valve,2000
1,20,4.99,Valve,1999
2,30,4.99,Valve,2003
3,40,4.99,Valve,2001
4,50,4.99,Gearbox Software,1999
...,...,...,...,...
10973,375450,59.99,"KOEI TECMO GAMES CO., LTD.",2016
10974,353390,NaN,Sin dato disponible,Sin dato disponible
10975,354280,5.99,CashcowGames,2016
10976,433920,14.99,"Amaranth Games, LLC",2016


In [9]:
utils.verificar_tipo_datos(df_items_developer)

,nombre_campo,tipo_datos,no_nulos_%,nulos_%,nulos
0,item_id,[<class 'int'>],100.0,0.0,0
1,developer,[<class 'str'>],100.0,0.0,0
2,release_anio,[<class 'str'>],100.0,0.0,0


## userdata

In [10]:
def userdata(user_id):
    usuario = df_reviews[df_reviews['user_id'] == user_id]
    cantidad_dinero = usuario['price'].sum()
    total_recomendaciones = usuario['reviews_recommend'].sum()
    total_items = usuario['items_count'].sum()
    
    total_reviews = len(df_reviews['user_id'].unique())
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': total_items.astype(int)
    }

In [103]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 254.88,
 'porcentaje_recomendacion': 0.09,
 'total_items': 529}

## countreviews

In [12]:
def countreviews(fecha_inicio, fecha_fin):
      
    user_data_entre_fechas = df_reviews[(df_reviews['reviews_date'] >= fecha_inicio) & (df_reviews['reviews_date'] <= fecha_fin)]
    total_usuarios = user_data_entre_fechas['user_id'].nunique()

    total_recomendacion = user_data_entre_fechas['reviews_recommend'].sum()
    porcentaje_recomendaciones = (total_usuarios / total_recomendacion) * 100
    
    return {
        'total_usuarios_reviews': total_usuarios,
        'porcentaje_recomendaciones': round(porcentaje_recomendaciones,2)
    }

In [13]:
countreviews('2011-11-05', '2012-12-24')

{'total_usuarios_reviews': 926, 'porcentaje_recomendaciones': 37.95}

### genre

In [14]:
def genre(genero):
    rank = genre_ranking[genre_ranking['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [15]:
genero = 'Simulation'
genre(genero)

{'rank': 5}

### userforgenre

In [16]:
def userforgenre(genero):
    data_por_genero = df_reviews[df_reviews['genres'] == genero]
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    top_users_dict = {}
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url'],
            #'playtime_horas': row['playtime_horas']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [17]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Steamified',
  'user_url': 'http://steamcommunity.com/id/Steamified'},
 2: {'user_id': 'idonothack',
  'user_url': 'http://steamcommunity.com/id/idonothack'},
 3: {'user_id': 'Efreak', 'user_url': 'http://steamcommunity.com/id/Efreak'},
 4: {'user_id': 'BlackReaperHei',
  'user_url': 'http://steamcommunity.com/id/BlackReaperHei'},
 5: {'user_id': '76561198058230663',
  'user_url': 'http://steamcommunity.com/profiles/76561198058230663'}}

### developer

In [105]:
df_items_developer.head(20)

,item_id,price,developer,release_anio
0,10,9.99,Valve,2000
1,20,4.99,Valve,1999
2,30,4.99,Valve,2003
3,40,4.99,Valve,2001
4,50,4.99,Gearbox Software,1999
5,60,4.99,Valve,2000
6,70,9.99,Valve,1998
7,130,4.99,Gearbox Software,2001
8,300,9.99,Valve,2010
9,240,19.99,Valve,2004


In [43]:
df_items_developer['developer'].value_counts()

Sin dato disponible        2373
Humongous Entertainment      35
Telltale Games               31
MumboJumbo                   30
Choice of Games              29
                           ... 
Mechanical Boss               1
Skatanic Studios              1
Robot Loves Kitty             1
Blam! Games LLC               1
CashcowGames                  1
Name: developer, Length: 5654, dtype: int64

In [96]:
def developer(desarrollador):
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    
    free_contenido_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio').size()
    total_contenido_año = data_filtrada.groupby('release_anio').size()

    porcentaje = (free_contenido_año / total_contenido_año * 100).fillna(0).astype(int)

    result_dict = dict(zip(porcentaje.index.astype(str), porcentaje.values))
    
    return result_dict

In [104]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'2016': 100}

## sentiment_analysis

In [108]:
df_reviews.head(10)

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,items_count,price,genres,release_anio,developer,playtime_horas
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,277.0,19.99,Action,2009,Tripwire Interactive,5060.666667
1,js41637,http://steamcommunity.com/id/js41637,251610,15 of 20 people (75%) found this review helpful,True,2014-06-24,1,888.0,NaN,Sin dato disponible,Sin dato disponible,Sin dato disponible,5539.916667
2,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Action,2013,"Hopoo Games, LLC",6180.266667
3,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,Indie,2013,"Hopoo Games, LLC",6180.266667
4,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,Formato inválido,2,137.0,9.99,RPG,2013,"Hopoo Games, LLC",6180.266667
5,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013-10-14,2,541.0,24.99,Adventure,2013,Telltale Games,7633.366667
6,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014-04-15,1,629.0,19.99,Action,2012,FromSoftware,14165.433333
7,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014-04-15,1,629.0,19.99,RPG,2012,FromSoftware,14165.433333
8,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014-05-05,0,NaN,19.99,Action,2013,Traveller's Tales,NaN
9,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014-05-05,0,NaN,19.99,Adventure,2013,Traveller's Tales,NaN


In [106]:
def sentiment_analysis(anio):
    # Filtrar las reseñas del desarrollador específico
    anio_reviews = df_reviews[df_reviews['release_anio'] == anio]
    
    # Inicializar un diccionario para contar las categorías de sentimiento
    sentiment_counts = {'Negative': 0, 'Neutral': 0, 'Positive': 0}
    
    # Iterar a través de las reseñas del desarrollador
    for index, row in anio_reviews.iterrows():
        sentiment = row['sentiment_analysis']
        sentiment_category = ''
        
        # Asignar la categoría de sentimiento correspondiente
        if sentiment == 0:
            sentiment_category = 'Negative'
        elif sentiment == 1:
            sentiment_category = 'Neutral'
        elif sentiment == 2:
            sentiment_category = 'Positive'
        
        # Incrementar el contador correspondiente en el diccionario
        sentiment_counts[sentiment_category] += 1
    
    return sentiment_counts

In [101]:
anio = 2000
sentiment_analysis(anio
                   )

{'Negative': 969, 'Neutral': 7783, 'Positive': 4817}